# Setup

In [ ]:
if False:
    !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl
    import torch_xla
    import torch_xla.core.xla_model as xm

In [ ]:
import sys
sys.path.append('../input/hotel-id-starter-similarity-training/pytorch-image-models/')
sys.path.append('../input/timm-pretrained-efficientnet/efficientnet/')

# Imports

In [ ]:
import numpy as np
import pandas as pd
import random
import os
import math

!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp '../input/timm-pretrained-efficientnet/efficientnet/efficientnet_b1-533bc792.pth' '/root/.cache/torch/hub/checkpoints/efficientnet_b1-533bc792.pth'

In [ ]:
from PIL import Image as pil_image
from tqdm import tqdm

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import timm
from sklearn.metrics.pairwise import cosine_similarity

# Global

In [ ]:
SEED = 42
IMG_SIZE = 512 #256
N_MATCHES = 5

PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
TRAIN_DATA_FOLDER = f"../input/hotelid-2022-train-images-{IMG_SIZE}x{IMG_SIZE}/images/"
TEST_DATA_FOLDER = PROJECT_FOLDER + "test_images/"

In [ ]:
print(os.listdir(PROJECT_FOLDER))

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Dataset and transformations

In [ ]:
import albumentations as A
import albumentations.pytorch as APT
import cv2 

base_transform = A.Compose([
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

In [ ]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
    return img

def open_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = pad_image(img)
    return cv2.resize(img, (IMG_SIZE, IMG_SIZE))

In [ ]:
class HotelImageDataset:
    def __init__(self, data, transform=None, data_folder="train_images/"):
        self.data = data
        self.data_folder = data_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_folder + record["image_id"]
        
        image = np.array(open_and_preprocess_image(image_path)).astype(np.uint8)

        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]
        
        return {
            "image" : image,
        }

# Model

In [ ]:
# class EmbeddingModel(nn.Module):
#     def __init__(self, n_classes=100, embedding_size=64, backbone_name="efficientnet_b0"):
#         super(EmbeddingModel, self).__init__()
        
#         self.backbone = timm.create_model(backbone_name, num_classes=n_classes, pretrained=False)
#         in_features = self.backbone.get_classifier().in_features
        
#         self.backbone.classifier = nn.Identity()
#         self.embedding = nn.Linear(in_features, embedding_size)
#         self.classifier = nn.Linear(embedding_size, n_classes)

#     def embed_and_classify(self, x):
#         x = self.forward(x)
#         return x, self.classifier(x)

#     def forward(self, x):
#         x = self.backbone(x)
#         x = x.view(x.size(0), -1)
#         x = self.embedding(x)
#         return x

# source: https://github.com/ronghuaiyang/arcface-pytorch/blob/master/models/metrics.py
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi-m)
        self.mm = math.sin(math.pi-m)*m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0-torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine*self.cos_m-sine*self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine>0, phi, cosine)
        else:
            phi = torch.where(cosine>self.th, phi, cosine-self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device=args.device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot*phi)+((1.0-one_hot)*cosine)
        output *= self.s

        return output

class EmbeddingModel(nn.Module):
    def __init__(self, out_features=100, embed_size=256, backbone_name="efficientnet_b1"):
        super(EmbeddingModel, self).__init__()

        self.embed_size = embed_size
        self.backbone = timm.create_model(backbone_name, pretrained=True)
        in_features = self.backbone.get_classifier().in_features
        self.embedding = nn.Linear(in_features, embed_size)
        self.classifier = nn.Linear(embed_size, out_features)

        '''fc_name, _ = list(self.backbone.named_modules())[-1]
        if fc_name == 'classifier':
            self.backbone.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.backbone.fc = nn.Identity()
        else:
            raise Exception("unknown classifier layer: "+fc_name)'''
        self.backbone.classifier = nn.Identity()

        self.arc_face = ArcMarginProduct(self.embed_size, out_features, s=30.0, m=0.20, easy_margin=False)

        self.post = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.embed_size*2), dim=None),
            nn.BatchNorm1d(self.embed_size*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.embed_size*2, self.embed_size)),
            nn.BatchNorm1d(self.embed_size),
        )

        print(f"Model {backbone_name} ArcMarginProduct - Features: {in_features}, Embeds: {self.embed_size}")
        
    def embed_and_classify(self, x):
        x = self.forward(x)
        return x, self.classifier(x)
    
    def forward(self, input, targets=None):
        x = self.backbone(input)
        x = x.view(x.size(0), -1)
        x = self.embedding(x) #self.post(x)
        
        if targets is not None:
            logits = self.arc_face(x, targets)
            return x, logits
        
        return x

# Model helper functions

In [ ]:
def generate_embeddings(args, loader, model, bar_desc="Generating embeds"):
    outputs_all = []
    
    model.eval()
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            output = model(input)
            outputs_all.extend(output.detach().cpu().numpy())
            
    return outputs_all

In [ ]:
def find_matches_agg(query, base_embeds, base_targets, k=N_MATCHES, expfrom=30, topk=100):
    distance_df = pd.DataFrame(index=np.arange(len(base_targets)), data={"hotel_id": base_targets})
    # calculate cosine distance of query embeds to all base embeds
    distance_df["distance"] = cosine_similarity([query], list(base_embeds))[0]
    # sort by distance and hotel_id
    distance_df = distance_df.sort_values(by=["distance", "hotel_id"], ascending=False).reset_index(drop=True)
    distance_df = distance_df[:topk]
    distance_df['rankscore'] = np.exp(np.linspace(expfrom,0,topk))/np.exp(expfrom) * np.maximum(0, distance_df["distance"])
    hotel_rankscore = distance_df.groupby('hotel_id')['rankscore'].agg(['sum', 'count'])
    hotel_rankscore = hotel_rankscore.sort_values(by=["sum"], ascending=False).reset_index()
    return hotel_rankscore["hotel_id"].values[:N_MATCHES]
    
def find_matches(query, base_embeds, base_targets, k=N_MATCHES):
    distance_df = pd.DataFrame(index=np.arange(len(base_targets)), data={"hotel_id": base_targets})
    # calculate cosine distance of query embeds to all base embeds
    distance_df["distance"] = cosine_similarity([query], list(base_embeds))[0]
    # sort by distance and hotel_id
    distance_df = distance_df.sort_values(by=["distance", "hotel_id"], ascending=False).reset_index(drop=True)
    # return first 5 different hotel_id_codes
    return distance_df["hotel_id"].unique()[:N_MATCHES]


def predict(args, base_embeddings_df, test_loader, model):
    test_embeds = generate_embeddings(args, test_loader, model, "Generate test embeddings")
    
    preds = []
    for query_embeds in tqdm(test_embeds, desc="Similarity - match finding"):
        tmp = find_matches(query_embeds, 
                           base_embeddings_df["embeddings"].values, 
                           base_embeddings_df["hotel_id"].values)
        preds.extend([tmp])
        
    return preds

# Prepare data

In [ ]:
test_df = pd.DataFrame(data={"image_id": os.listdir(TEST_DATA_FOLDER), "hotel_id": ""}).sort_values(by="image_id")

# Prepare model

In [ ]:
def get_model(backbone_name, checkpoint_path, args):
    model = EmbeddingModel(args.n_classes, args.embedding_size, backbone_name)
        
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint["model"])
    model = model.to(args.device)
    
    return model

In [ ]:
class args:
    batch_size = 4 #64
    num_workers = 2
    embedding_size = 256 #128
    device = ('cuda' if torch.cuda.is_available() else 'cpu')

seed_everything(seed=SEED)

test_dataset = HotelImageDataset(test_df, base_transform, data_folder=TEST_DATA_FOLDER)
test_loader  = DataLoader(test_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

In [ ]:
from glob import glob 
epoch = 8
# run_dir = "hotel-id-starter-similarity-training"
run_dir = "argfacecheckpoints"
# run_dir = "clawsctadawfbnetc-100"
# run_dir = "hotel-id-2022-tpu-cpu-conversion"
# run_dir = "dataset-hotelid-2022-tpucpu-conversion"

# backbone_name = "efficientnet_b0"
backbone_name = "efficientnet_b1"
#backbone_name = "fbnetc_100"
checkpointfile = glob(f"../input/{run_dir}/checkpoint-embedding-model-{backbone_name}-{IMG_SIZE}x{IMG_SIZE}/epoch{epoch}_*")[0]
print(checkpointfile)
base_embeddings_df = pd.read_pickle(f'../input/{run_dir}/embedding-model-{backbone_name}-{IMG_SIZE}x{IMG_SIZE}_image-embeddings/epoch{epoch}.pkl')

In [ ]:
args.n_classes = base_embeddings_df["hotel_id"].nunique()

In [ ]:
model = get_model(backbone_name,checkpointfile, args)

# Submission

In [ ]:
%%time

preds = predict(args, base_embeddings_df, test_loader, model)
# transform array of hotel_ids into string
test_df["hotel_id"] = [str(list(l)).strip("[]").replace(",", "") for l in preds]

test_df.to_csv("submission.csv", index=False)
test_df.head()